Experimento 2 - KNN Imputer, 3sigma outlier detection, min max normalization, MLP classifier

In [14]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import seaborn.objects as so
from ucimlrepo import fetch_ucirepo

In [15]:
beans = fetch_ucirepo(id=602)
df = beans.data.features
targets = beans.data.targets

In [16]:
cols = ['Area', 'Perimeter', 'MajorAxisLength', 'MinorAxisLength', 'AspectRatio', 'Eccentricity', 'ConvexArea', 'EquivDiameter', 'Extent', 'Solidity', 'Roundness', 'Compactness', 'ShapeFactor1', 'ShapeFactor2', 'ShapeFactor3', 'ShapeFactor4']

In [17]:
import random

#Introducing Missing values (5%)
x = df.copy()

for index, i in enumerate(df):
  for jndex, j in enumerate(df[i]):
    if random.randint(0,100) < 5:
      x[i][jndex] = np.NaN

df = x

/tmp/ipykernel_8842/4206422893.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x[i][jndex] = np.NaN
/tmp/ipykernel_8842/4206422893.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x[i][jndex] = np.NaN
/tmp/ipykernel_8842/4206422893.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x[i][jndex] = np.NaN
/tmp/ipykernel_8842/4206422893.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

In [18]:
df.isna().sum()

Area               639
Perimeter          681
MajorAxisLength    701
MinorAxisLength    666
AspectRatio        662
Eccentricity       664
ConvexArea         652
EquivDiameter      643
Extent             677
Solidity           662
Roundness          676
Compactness        698
ShapeFactor1       663
ShapeFactor2       698
ShapeFactor3       635
ShapeFactor4       684
dtype: int64

In [19]:
df

,Area,Perimeter,MajorAxisLength,MinorAxisLength,AspectRatio,Eccentricity,ConvexArea,EquivDiameter,Extent,Solidity,Roundness,Compactness,ShapeFactor1,ShapeFactor2,ShapeFactor3,ShapeFactor4
0,28395.0,610.291,NaN,173.888747,1.197191,0.549812,28715.0,190.141097,NaN,0.988856,0.958027,0.913358,0.007332,0.003147,0.834222,0.998724
1,28734.0,638.018,200.524796,182.734419,1.097356,0.411785,29172.0,191.272751,0.783968,0.984986,0.887034,0.953861,0.006979,0.003564,0.909851,0.998430
2,29380.0,624.110,212.826130,175.931143,1.209713,0.562727,29690.0,193.410904,0.778113,0.989559,0.947849,0.908774,0.007244,0.003048,NaN,0.999066
3,30008.0,645.884,210.557999,182.516516,1.153638,0.498616,30724.0,195.467062,0.782681,0.976696,0.903936,0.928329,0.007017,0.003215,0.861794,0.994199
4,30140.0,620.134,201.847882,190.279279,1.060798,0.333680,30417.0,195.896503,0.773098,0.990893,0.984877,0.970516,0.006697,NaN,0.941900,0.999166
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13606,42097.0,759.696,288.721612,185.944705,1.552728,NaN,42508.0,231.515799,0.714574,0.990331,0.916603,0.801865,0.006858,0.001749,0.642988,0.998385
13607,42101.0,NaN,281.576392,190.713136,1.476439,0.735702,42494.0,231.526798,0.799943,0.990752,0.922015,0.822252,0.006688,0.001886,NaN,0.998219
13608,42139.0,759.321,281.539928,191.187979,1.472582,0.734065,42569.0,231.631261,0.729932,0.989899,0.918424,0.822730,0.006681,0.001888,0.676884,0.996767
13609,42147.0,763.779,283.382636,190.275731,1.489326,0.741055,42667.0,231.653247,0.705389,0.987813,0.907906,0.817457,0.006724,0.001852,0.668237,0.995222


In [20]:
#Imputing Missing Values with KNN Imputer
from sklearn.impute import KNNImputer as knni

imputer = knni(n_neighbors=5)
df_imputed = pd.DataFrame(imputer.fit_transform(df), columns = df.columns)
df_imputed

,Area,Perimeter,MajorAxisLength,MinorAxisLength,AspectRatio,Eccentricity,ConvexArea,EquivDiameter,Extent,Solidity,Roundness,Compactness,ShapeFactor1,ShapeFactor2,ShapeFactor3,ShapeFactor4
0,28395.0,610.2910,231.792482,173.888747,1.197191,0.549812,28715.0,190.141097,0.761419,0.988856,0.958027,0.913358,0.007332,0.003147,0.834222,0.998724
1,28734.0,638.0180,200.524796,182.734419,1.097356,0.411785,29172.0,191.272751,0.783968,0.984986,0.887034,0.953861,0.006979,0.003564,0.909851,0.998430
2,29380.0,624.1100,212.826130,175.931143,1.209713,0.562727,29690.0,193.410904,0.778113,0.989559,0.947849,0.908774,0.007244,0.003048,0.685330,0.999066
3,30008.0,645.8840,210.557999,182.516516,1.153638,0.498616,30724.0,195.467062,0.782681,0.976696,0.903936,0.928329,0.007017,0.003215,0.861794,0.994199
4,30140.0,620.1340,201.847882,190.279279,1.060798,0.333680,30417.0,195.896503,0.773098,0.990893,0.984877,0.970516,0.006697,0.002377,0.941900,0.999166
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13606,42097.0,759.6960,288.721612,185.944705,1.552728,0.757389,42508.0,231.515799,0.714574,0.990331,0.916603,0.801865,0.006858,0.001749,0.642988,0.998385
13607,42101.0,756.1986,281.576392,190.713136,1.476439,0.735702,42494.0,231.526798,0.799943,0.990752,0.922015,0.822252,0.006688,0.001886,0.695725,0.998219
13608,42139.0,759.3210,281.539928,191.187979,1.472582,0.734065,42569.0,231.631261,0.729932,0.989899,0.918424,0.822730,0.006681,0.001888,0.676884,0.996767
13609,42147.0,763.7790,283.382636,190.275731,1.489326,0.741055,42667.0,231.653247,0.705389,0.987813,0.907906,0.817457,0.006724,0.001852,0.668237,0.995222


In [21]:
#adding the labels
df_imputed['Class'] = targets
df_pre_missing_values = df.copy()
print(df_imputed.groupby('Class').agg(['count']))
df_imputed['Class'] = df_imputed['Class'].transform(lambda x: 0 if x == 'BARBUNYA' else (1 if x == 'BOMBAY' else (2 if x == 'CALI' else (3 if x == 'DERMASON' else (4 if x == 'HOROZ' else (5 if x == 'SEKER' else 6))))))

          Area Perimeter MajorAxisLength MinorAxisLength AspectRatio  \
         count     count           count           count       count   
Class                                                                  
BARBUNYA  1322      1322            1322            1322        1322   
BOMBAY     522       522             522             522         522   
CALI      1630      1630            1630            1630        1630   
DERMASON  3546      3546            3546            3546        3546   
HOROZ     1928      1928            1928            1928        1928   
SEKER     2027      2027            2027            2027        2027   
SIRA      2636      2636            2636            2636        2636   

         Eccentricity ConvexArea EquivDiameter Extent Solidity Roundness  \
                count      count         count  count    count     count   
Class                                                                      
BARBUNYA         1322       1322          1322   13

In [22]:
#Outlier Removal with 3sigma
from scipy import stats


print(f'Pre Outlier Shape: {df_imputed.shape}')
df_no_outliers = df_imputed.copy()

print(df_no_outliers['Class'].unique())
for i in df_no_outliers['Class'].unique():
    class_unique = df_no_outliers[df_no_outliers['Class'] == i]
    for feature in class_unique:
      upper = class_unique[feature].mean() + (3 * class_unique[feature].std())
      lower = class_unique[feature].mean() - (3 * class_unique[feature].std())
      excluded = pd.Series(class_unique[class_unique[feature] < lower].index)
      #print(excluded.values)
      df_no_outliers.drop(excluded.values, inplace = True, errors='ignore')

print(df_no_outliers.groupby('Class').count())
print(f'Pos Outlier Shape: {df_no_outliers.shape}')


Pre Outlier Shape: (13611, 17)
[5 0 1 2 4 6 3]
       Area  Perimeter  MajorAxisLength  MinorAxisLength  AspectRatio  \
Class                                                                   
0      1257       1257             1257             1257         1257   
1       486        486              486              486          486   
2      1576       1576             1576             1576         1576   
3      3405       3405             3405             3405         3405   
4      1800       1800             1800             1800         1800   
5      1915       1915             1915             1915         1915   
6      2536       2536             2536             2536         2536   

       Eccentricity  ConvexArea  EquivDiameter  Extent  Solidity  Roundness  \
Class                                                                         
0              1257        1257           1257    1257      1257       1257   
1               486         486            486     486    

In [23]:
df_no_outliers['Class'] = df_no_outliers['Class'].transform(lambda x: 'BARBUNYA' if x == 0 else ('BOMBAY' if x == 1 else ('CALI' if x == 2 else ('DERMASON' if x == 3 else ('HOROZ' if x == 4 else ('SEKER' if x == 5 else 'SIRA'))))))
df_no_outliers = df_no_outliers.reset_index()
print(df_no_outliers.groupby('Class').agg(['count']))

         index  Area Perimeter MajorAxisLength MinorAxisLength AspectRatio  \
         count count     count           count           count       count   
Class                                                                        
BARBUNYA  1257  1257      1257            1257            1257        1257   
BOMBAY     486   486       486             486             486         486   
CALI      1576  1576      1576            1576            1576        1576   
DERMASON  3405  3405      3405            3405            3405        3405   
HOROZ     1800  1800      1800            1800            1800        1800   
SEKER     1915  1915      1915            1915            1915        1915   
SIRA      2536  2536      2536            2536            2536        2536   

         Eccentricity ConvexArea EquivDiameter Extent Solidity Roundness  \
                count      count         count  count    count     count   
Class                                                              

In [24]:
df_no_outliers = df_no_outliers.drop(['index'], axis='columns')
df_no_outliers

,Area,Perimeter,MajorAxisLength,MinorAxisLength,AspectRatio,Eccentricity,ConvexArea,EquivDiameter,Extent,Solidity,Roundness,Compactness,ShapeFactor1,ShapeFactor2,ShapeFactor3,ShapeFactor4,Class
0,28395.0,610.2910,231.792482,173.888747,1.197191,0.549812,28715.0,190.141097,0.761419,0.988856,0.958027,0.913358,0.007332,0.003147,0.834222,0.998724,SEKER
1,28734.0,638.0180,200.524796,182.734419,1.097356,0.411785,29172.0,191.272751,0.783968,0.984986,0.887034,0.953861,0.006979,0.003564,0.909851,0.998430,SEKER
2,29380.0,624.1100,212.826130,175.931143,1.209713,0.562727,29690.0,193.410904,0.778113,0.989559,0.947849,0.908774,0.007244,0.003048,0.685330,0.999066,SEKER
3,30279.0,634.9270,212.560556,181.510182,1.171067,0.520401,30600.0,193.117309,0.775688,0.989510,0.943852,0.923726,0.007898,0.003153,0.853270,0.999236,SEKER
4,30477.0,670.0330,211.050155,184.039050,1.146768,0.489478,30970.0,196.988633,0.762402,0.984081,0.853080,0.933374,0.006925,0.003242,0.871186,0.999049,SEKER
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12970,42097.0,759.6960,288.721612,185.944705,1.552728,0.757389,42508.0,231.515799,0.714574,0.990331,0.916603,0.801865,0.006858,0.001749,0.642988,0.998385,DERMASON
12971,42101.0,756.1986,281.576392,190.713136,1.476439,0.735702,42494.0,231.526798,0.799943,0.990752,0.922015,0.822252,0.006688,0.001886,0.695725,0.998219,DERMASON
12972,42139.0,759.3210,281.539928,191.187979,1.472582,0.734065,42569.0,231.631261,0.729932,0.989899,0.918424,0.822730,0.006681,0.001888,0.676884,0.996767,DERMASON
12973,42147.0,763.7790,283.382636,190.275731,1.489326,0.741055,42667.0,231.653247,0.705389,0.987813,0.907906,0.817457,0.006724,0.001852,0.668237,0.995222,DERMASON


In [25]:
#Normalization with min max
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
scaler.fit_transform(df_no_outliers[cols])
df_scaled = pd.DataFrame(scaler.transform(df_no_outliers[cols]), columns = cols)

df_scaled['Class'] = df_no_outliers['Class']
df_scaled

,Area,Perimeter,MajorAxisLength,MinorAxisLength,AspectRatio,Eccentricity,ConvexArea,EquivDiameter,Extent,Solidity,Roundness,Compactness,ShapeFactor1,ShapeFactor2,ShapeFactor3,ShapeFactor4,Class
0,0.034338,0.060868,0.084608,0.134028,0.096658,0.325651,0.033564,0.071030,0.662971,0.779503,0.900083,0.831986,0.639626,0.861196,0.803502,0.961646,SEKER
1,0.035806,0.080752,0.026737,0.160782,0.023522,0.068251,0.035495,0.073829,0.735504,0.632906,0.682882,0.955521,0.590062,1.000000,0.946861,0.950484,SEKER
2,0.038603,0.070778,0.049504,0.140205,0.105831,0.349735,0.037684,0.079116,0.716671,0.806121,0.868945,0.818006,0.627322,0.828005,0.521265,0.974660,SEKER
3,0.042495,0.078535,0.049013,0.157079,0.077520,0.270803,0.041529,0.078390,0.708871,0.804267,0.856715,0.863609,0.719207,0.863027,0.839608,0.981111,SEKER
4,0.043352,0.103711,0.046217,0.164728,0.059720,0.213136,0.043093,0.087964,0.666131,0.598657,0.579002,0.893035,0.582510,0.892778,0.873570,0.973999,SEKER
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12970,0.093662,0.168010,0.189975,0.170492,0.357114,0.712750,0.091848,0.173345,0.512286,0.835379,0.773349,0.491932,0.573181,0.395047,0.441003,0.948770,DERMASON
12971,0.093680,0.165502,0.176750,0.184915,0.301227,0.672307,0.091788,0.173373,0.786890,0.851303,0.789907,0.554113,0.549250,0.440635,0.540970,0.942435,DERMASON
12972,0.093844,0.167742,0.176683,0.186351,0.298401,0.669254,0.092105,0.173631,0.561689,0.818998,0.778920,0.555569,0.548281,0.441448,0.505256,0.887246,DERMASON
12973,0.093879,0.170938,0.180093,0.183592,0.310667,0.682289,0.092519,0.173685,0.482741,0.739982,0.746742,0.539489,0.554244,0.429363,0.488864,0.828503,DERMASON


In [26]:
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import cross_validate


x_train = df_scaled.iloc[:, 0:16]
y_train = df_scaled.iloc[:, 16]
classifier = MLPClassifier(activation='logistic', solver='adam', alpha=1e-5, hidden_layer_sizes=(12, 3), random_state=1, verbose=True, learning_rate_init=0.3, tol=1e-3, max_iter=500)
scoring = {'acc' : 'accuracy',
           'prec' : 'precision_macro',
           'recall' : 'recall_macro',
           'f1' : 'f1_macro'}


y_pred = cross_validate(classifier, x_train, y_train, cv=10, scoring=scoring, return_train_score=True)
print('Experimento 2')
print('Acurácia Média: ' + '%.2f' % (np.mean(y_pred['test_acc'])*100) + '%')
print('Precisão Média: ' + '%.2f' % (np.mean(y_pred['test_prec'])*100) + '%')
print('Revocação Média: ' + '%.2f' % (np.mean(y_pred['test_recall'])*100) + '%')
print('F1-Score Médio: ' + '%.2f' % (np.mean(y_pred['test_f1'])*100) + '%')


Iteration 1, loss = 1.35166134
Iteration 2, loss = 1.02516723
Iteration 3, loss = 0.95627146
Iteration 4, loss = 0.89101972
Iteration 5, loss = 0.86131644
Iteration 6, loss = 0.81846317
Iteration 7, loss = 0.77910835
Iteration 8, loss = 0.75874963
Iteration 9, loss = 0.72591749
Iteration 10, loss = 0.71115490
Iteration 11, loss = 0.71547739
Iteration 12, loss = 0.69768027
Iteration 13, loss = 0.69686417
Iteration 14, loss = 0.67607723
Iteration 15, loss = 0.72794434
Iteration 16, loss = 0.68692390
Iteration 17, loss = 0.65953463
Iteration 18, loss = 0.63949065
Iteration 19, loss = 0.64802769
Iteration 20, loss = 0.62139219
Iteration 21, loss = 0.61922989
Iteration 22, loss = 0.62245181
Iteration 23, loss = 0.60802165
Iteration 24, loss = 0.58571967
Iteration 25, loss = 0.58875529
Iteration 26, loss = 0.59833350
Iteration 27, loss = 0.78002307
Iteration 28, loss = 0.60457294
Iteration 29, loss = 0.57320482
Iteration 30, loss = 0.58709462
Iteration 31, loss = 0.59194374
Iteration 32, los